# Atividade 4

## Equipe:

* Felipe Getúlio Laranjeira do Nascimento
* Lucas Pereira Reis

## Importação das bibliotecas

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import combination as comb
from math import *

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier

## Realizando a leitura do *dataset*

In [74]:
df = pd.read_csv('seeds_dataset.txt', sep='\t', header=None)
df.columns = ["area","perimeter","compactness",
              "length_kernel","width_kernel","asymmetry_coefficient",
              "length_kernel_groove","variety"]

In [75]:
df.head()

,area,perimeter,compactness,length_kernel,width_kernel,asymmetry_coefficient,length_kernel_groove,variety
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


#### Observação:
Se tu pensar em um gráfico legal pra mostrar isso, pode colocar

In [76]:
target = df.variety
qtd = []
for i in target.unique():
    qtd.append(len(df.loc[df.variety == i]))

print(f'Quantidade de amostras da classe Karma: {qtd[0]}')
print(f'Quantidade de amostras da classe Rosa: {qtd[1]}')
print(f'Quantidade de amostras da classe Canadian: {qtd[2]}')

Quantidade de amostras da classe Karma: 70
Quantidade de amostras da classe Rosa: 70
Quantidade de amostras da classe Canadian: 70


Como pode ser visto, o *dataset* é igualmente distribuído, assim para a métrica `f1_score` será utilizado o paramêtro de `macro_averaging`

# Pré-Processamento de Dados

In [77]:
df.drop(['variety'],axis=1,inplace=True)

In [78]:
n_i = len(df.columns) #numero de neuronios na camada de entrada
n_o = 3 #numero de neuronios na camada de saida (sujeito a mudanças)

In [79]:
def geometric_pyramid(alpha):
    return alpha*sqrt(n_i*n_o)

In [85]:
X_train, Y_train, X_test, Y_test = train_test_split(df,target,test_size=0.3)

#### Observação:
Para um `alpha = 2` ou `alpha = 3` o algoritmo para gerar o subconjunto fica bastante pesado. Talvez seja necessário diminuir o valor de `n_i` retirando colunas desnecessárias ou utilizar PCA. Coloquei `alpha = 1` apenas para testar

In [81]:
alpha = [0.5,1]
n_h = [floor(geometric_pyramid(a)) for a in alpha]

hidden_layer_sizes = []

for n in n_h:
    subsets = comb.partitions(n)
    hidden_layer_sizes = hidden_layer_sizes + subsets

# Paramêtros para a busca em grade

#### Observação:
Professora pediu pra explicar porquê utilizou tal `solver` para a rede neural. Ela disse que apenas um serviria para o projeto, precisa-se procurar saber qual é

In [82]:
params = {
    'activation': ['identity', 'logistic','tanh','relu'],
    'hidden_layer_sizes': hidden_layer_sizes,
    'max_iter': [400],
    'solver': ['sgd']
}

# Busca em Grade

In [83]:
gs = GridSearchCV(MLPClassifier(), params, cv=3, scoring='f1_macro')

#### Observação:
Não tenho certeza se tem que colocar o dataset todo no `fit` do `gs`, tem que verificar